Step 1: Setup

We need to specify:

How often the bot runs

Which market data it uses

In [ ]:
from datetime import datetime, timedelta

from investing_algorithm_framework import (
    CCXTOHLCVMarketDataSource,
    CCXTTickerMarketDataSource
)

bitvavo_btc_eur_ohlcv_2h = CCXTOHLCVMarketDataSource(
    identifier="BTC/EUR-ohlcv",
    market="BITVAVO",
    symbol="BTC/EUR",
    timeframe="2h",
    # retrieve data from the last ~17 days
    start_date_func=lambda: datetime.utcnow() - timedelta(days=17)
)

# Ticker data to track orders, trades and positions
bitvavo_btc_eur_ticker = CCXTTickerMarketDataSource(
    identifier="BTC/EUR-ticker",
    market="BITVAVO",
    symbol="BTC/EUR",
)


- Simulation sandbox -> backtrader (python package)
- TaLib -> RIS
- ccxt -> connect to binance, base DEX

Step 2: Trading Strategy

We implement a Golden Cross / Death Cross strategy:

Buy when fast MA crosses above slow MA

Sell when fast MA crosses below slow MA

In [ ]:
from investing_algorithm_framework import (
    TradingStrategy,
    TimeUnit,
    Algorithm,
    OrderSide
)

import tulipy as tp
import pandas as pd


def is_crossover(fast_series, slow_series):
    return (
        fast_series[-2] <= slow_series[-2] and
        fast_series[-1] > slow_series[-1]
    )


def is_crossunder(fast_series, slow_series):
    return (
        fast_series[-2] >= slow_series[-2] and
        fast_series[-1] < slow_series[-1]
    )


class GoldenCrossDeathCrossTradingStrategy(TradingStrategy):
    time_unit = TimeUnit.HOUR
    interval = 2

    market_data_sources = [
        "BTC/EUR-ohlcv",
        "BTC/EUR-ticker"
    ]

    symbols = ["BTC/EUR"]

    def apply_strategy(self, algorithm: Algorithm, market_data: dict):
        for symbol in self.symbols:
            target_symbol = symbol.split("/")[0]

            # Avoid multiple open orders
            if algorithm.has_open_orders(target_symbol):
                continue

            ohlcv_data = market_data[f"{symbol}-ohlcv"]

            df = pd.DataFrame(
                ohlcv_data,
                columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]
            )

            fast = tp.sma(df["Close"].to_numpy(), period=9)
            slow = tp.sma(df["Close"].to_numpy(), period=50)

            price = market_data[f"{symbol}-ticker"]["bid"]

            if algorithm.has_position(target_symbol) and is_crossunder(fast, slow):
                algorithm.close_position(target_symbol)

            elif not algorithm.has_position(target_symbol) and is_crossover(fast, slow):
                algorithm.create_limit_order(
                    target_symbol=target_symbol,
                    order_side=OrderSide.BUY,
                    price=price,
                    percentage_of_portfolio=25,
                    precision=4
                )


Step 3: Backtest the Strategy

We now test the strategy using historical data.

In [ ]:
import sys
from datetime import datetime

from investing_algorithm_framework import (
    PortfolioConfiguration,
    pretty_print_backtest
)

from app import app


def convert_to_datetime(datetime_str):
    try:
        return datetime.strptime(datetime_str, "%Y-%m-%d")
    except ValueError:
        print(f"Error: Invalid datetime format for '{datetime_str}'. Use YYYY-MM-DD")
        sys.exit(1)


app.add_portfolio_configuration(
    PortfolioConfiguration(
        market="BITVAVO",
        trading_symbol="EUR",
        initial_balance=400
    )
)

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Error: Provide start_date and end_date")
        sys.exit(1)

    start_date = convert_to_datetime(sys.argv[1])
    end_date = convert_to_datetime(sys.argv[2])

    backtest_report = app.backtest(
        start_date=start_date,
        end_date=end_date,
        pending_order_check_interval="2h"
    )

    pretty_print_backtest(backtest_report)


Step 4: Analyze the Backtest Results

Example results:

Initial balance: 400 EUR

Final balance: ~468 EUR

Net gain: ~68 EUR

Growth rate: ~17%

Trades closed: 64

Positive trades: ~30%

Negative trades: ~70%

Despite low win rate, profitability comes from trend-following behavior.

# Deploy the bot - GCP / Azure

In [ ]:
import azure.functions as func

from investing_algorithm_framework import (
    StatelessAction,
    PortfolioConfiguration,
    MarketCredential
)

from app import app as trading_bot_app


trading_bot_app.add_portfolio_configuration(
    PortfolioConfiguration(
        market="BITVAVO",
        trading_symbol="EUR"
    )
)

trading_bot_app.add_market_credential(
    MarketCredential(
        market="BITVAVO",
        api_key="YOUR_BITVAVO_API_KEY",
        secret_key="YOUR_BITVAVO_SECRET_KEY"
    )
)

app = func.FunctionApp()


@app.timer_trigger(
    schedule="0 */2 * * *",
    arg_name="myTimer",
    run_on_startup=True,
    use_monitor=False
)
def trading_bot_azure_function(myTimer: func.TimerRequest) -> None:
    trading_bot_app.run(
        payload={"ACTION": StatelessAction.RUN_STRATEGY.value}
    )
